In [1]:
import os
import sys

cur_dir = os.getcwd()
pkg_rootdir = os.path.dirname(os.path.dirname(cur_dir))
if pkg_rootdir not in sys.path:
    sys.path.append(pkg_rootdir)

In [2]:
import pandas as pd
from torch_geometric import seed_everything
import torch
from Utils.mol2graph import data_process
from Utils.evaluate_gnn import model_test, model_val, EarlyStopping
from torch import nn
from Utils.models import AttentiveFPModel
import warnings

warnings.filterwarnings("ignore")

In [3]:
seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
att_paramas = {
    "hidden_dim": 128,
    "num_layers": 3,
    "dropout": 0.2,
    "num_timesteps": 10,
    "batch_size": 32,
    "lr": 0.001,
}

In [5]:
train = pd.read_excel("../../Dataset/train_data.xlsx", index_col=False)
test = pd.read_excel("../../Dataset/test_data.xlsx", index_col=False)
out_test = pd.read_excel("../../Dataset/data_out_feats.xlsx", index_col=False)

train_loader = data_process(
    train, batch_size=att_paramas["batch_size"], label_str="P", shuffle=True
)
test_loader = data_process(test, batch_size=len(test), label_str="P", shuffle=False)
out_test_loader = data_process(
    out_test, batch_size=len(out_test), label_str="P", shuffle=False
)

[20:56:41] WARNING: not removing hydrogen atom without neighbors
[20:56:41] WARNING: not removing hydrogen atom without neighbors


In [6]:
train_rmse_values = []  # 替换为训练集 RMSE 的记录值
val_rmse_values = []  # 替换为验证集 RMSE 的记录值
test_rmse_values = []  # 替换为测试集 RMSE 的记录值

In [ ]:
model = AttentiveFPModel(
    input_dim=72,
    hidden_dim=att_paramas["hidden_dim"],
    output_dim=1,
    num_layers=att_paramas["num_layers"],
    dropout=att_paramas["dropout"],
    num_timesteps=att_paramas["num_timesteps"],
).to(device)


model.load_state_dict(torch.load("./ATT_P_" + ".pth"))
train_r2, train_mae, train_rmse, train_y_true, train_y_pred = model_test(
    model, train_loader, device
)

test_r2, test_mae, test_rmse, test_y_true, test_y_pred = model_test(
    model, test_loader, device
)

out_r2, out_mae, out_rmse, out_y_true, out_y_pred = model_test(
    model, out_test_loader, device
)

train_rmse_values.append(train_rmse)

val_rmse_values.append(test_rmse)

test_rmse_values.append(out_rmse)

# print(f"Epoch {epoch:03d}:")

print(
    f"  Train      - R2: {train_r2:.4f}, RMSE: {train_rmse:.4f}, MAE: {train_mae:.4f}"
)

print(f"  Test      - R2: {test_r2:.4f}, RMSE: {test_rmse:.4f}, MAE: {test_mae:.4f}")
print(f"  Out       - R2: {out_r2:.4f}, RMSE: {out_rmse:.4f}, MAE: {out_mae:.4f}")

  Train      - R2: 0.8937, RMSE: 1.4135, MAE: 1.0079
  Test      - R2: 0.6871, RMSE: 2.3231, MAE: 1.5747
  Out       - R2: 0.8039, RMSE: 2.3288, MAE: 1.5643


In [8]:
train = {"P": train_y_true, "P_pred": train_y_pred}

test = {"test_y": test_y_true, "y_pred": test_y_pred}

out_test = {"out_y": out_y_true, "out_y_pred": out_y_pred}
metrics = {
    "train_r2": [train_r2],
    "test_r2": [test_r2],
    "out_r2": [out_r2],
    "train_rmse": [train_rmse],
    "test_rmse": [test_rmse],
    "out_rmse": [out_rmse],
    "train_mae": [train_mae],
    "test_mae": [test_mae],
    "out_mae": [out_mae],
}

train = pd.DataFrame(train)
test = pd.DataFrame(test)
out_test = pd.DataFrame(out_test)
metrics = pd.DataFrame(metrics)
train.to_excel("./results/P/ATT_train_data.xlsx", index=False)
test.to_excel("./results/P/ATT_test_data.xlsx", index=False)
out_test.to_excel("./results/P/ATT_out_test_data.xlsx", index=False)
metrics.to_excel("./results/P/ATT_metrics.xlsx", index=False)